In [1]:
from ast import parse
from concurrent.futures import ThreadPoolExecutor
import numpy as np
import cv2
import argparse
import glob
import os.path as osp
import matplotlib.pyplot as plt
from facenet_pytorch import MTCNN
from utils import *
import torch
import os

# torch.multiprocessing.set_start_method('spawn')
device = torch.device('cpu' if torch.cuda.is_available() else 'cuda')
face_detector = MTCNN(device=device)

fake_image_dir = "../../../../2_Deep_Learning/Dataset/facial_forgery/FF+/image/fake"
fake_video_dir = "../../../../2_Deep_Learning/Dataset/facial_forgery/FF+/manipulated_sequences"
real_image_dir = "../../../../2_Deep_Learning/Dataset/facial_forgery/FF+/image/real"
real_video_dir = "../../../../2_Deep_Learning/Dataset/facial_forgery/FF+/original_sequences"


def parse_args():
    parser = argparse.ArgumentParser(description="Fake Detection")
    parser.add_argument('--in_dir', default='', help='path to train data')
    parser.add_argument('--out_dir', default='out', help='path to test data')
    parser.add_argument('--num_thread', default=4, type=int, help='number of threads')
    parser.add_argument('--duration', default=15, type=int)
    return parser.parse_args()

def extract_frame(video_path):
    # 
    output_dir = out_dir
    duration = durationss

    # Đọc video với videoCapture
    video = cv2.VideoCapture(video_path)
    video_name = osp.basename(video_path).split('.')[0]

    print(video_name)
    print(video)

    success = True
    image = None
    id_frame = 0
    
    while success:
        # Đọc video 4 lần (4 frame), chỉ lấy frame cuối cùng, nếu fail thì kết thúc vòng lặp
        for i in range(duration):
            success, image = video.read()
            if not success:
                break
            
        print("Get here")
        print("Success: ", success)
        print("Image: \n", image)
        try:
            # Convert from BGR (CV) to RGB (Image)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        except:
            continue

        plt.imsave(osp.join(output_dir, video_name + '_' + str(id_frame) + '.jpg'), image, format='jpg')
        id_frame += 1
        

def extract_face(video_path: str, ext_margin=0.2):
    #
    # forgery_tech = video_path.split('/')[-4]
    output_dir = out_dir if out_dir != '' else real_image_dir
    duration = durationss
    # Read video:

    video = cv2.VideoCapture(video_path)
    video_name = osp.basename(video_path).split('.')[0]

    # print("Forgery Tech: ", forgery_tech)
    print("Video name: ", video_name)
    # if not osp.exists(osp.join(output_dir, forgery_tech)):
    #     os.mkdir(osp.join(output_dir, forgery_tech))
    #
    success = True
    image = None
    id_frame = 0
    while success:
        # Read <duration> times, only get the last success if successes for all <duration> times
        for _ in range(duration):
            success, image = video.read()
            if not success:
                break
        try:
            # Convert from BGR to RGB
            image = cv2.cvtColor(image, code=cv2.COLOR_BGR2RGB)
        except:
            continue

        # Detect face:
        face_pos, probs = face_detector.detect(image, landmarks=False) 
        # vis_face_img(image, face_pos[0], probs[0], landmark=None, vis=False, save=None)

        # Find the face with largest prob 
        try:
            face_pos = face_pos[np.argmax(probs)]
        except:
            continue
        
        # Extend face rectangle
        xmin, ymin, xmax, ymax = face_pos
        x, y, w, h = map(int, [xmin, ymin, xmax-xmin, ymax-ymin])
        extend_x, extend_y = int(w * ext_margin), int(h * ext_margin)
        xmin = max(0, x - extend_x)
        ymin = max(0, y - extend_y)
        xmax = min(image.shape[1], x + w + extend_x)
        ymax = min(image.shape[0], y + h + extend_y)

        face = image[ymin:ymax, xmin:xmax]
        # Save to output dir:
        plt.imsave(osp.join(output_dir, 'faceswap_' + video_name + '_' + str(id_frame) + '.jpg'), face, format='jpg')
        # plt.imsave(osp.join(output_dir, video_name + '_' + str(id_frame) + '.jpg'), face, format='jpg')

        id_frame += 1
    print("Number of taken frame: {}".format(id_frame))


# args = parse_args()
### Step 1: Giải quyết real:
#    +) Tách 800 video train/200 video test.
#    +) Extract ra image.
#    +) Đặt vào all/deepfake/f2f/fs/nt train/0_real - 800 và test/0_real - 200
### Step 2: Giải quyết fake:
#   +) Với mỗi technique fake, tách ra 800/200: train/1_df - test/1_df
#   +) Extract tương ứng cho từng technique
#   +) Gộp lại đặt trong all
in_dir = '/mnt/disk1/doan/phucnp/Dataset/ff/video/manipulated_sequences/FaceSwap/train'
out_dir = "/mnt/disk1/doan/phucnp/Dataset/ff/video/manipulated_sequences/FaceSwap/image/train"
durationss = 25

if __name__ == '__main__':
    video_paths = []
    video_types = ['/*.mp4', '/*/*/*/*.avi']  # Deepfakes/c23/videos/*.mp4

    # Duyệt tất cả các path tới video
    for type in video_types:
        paths = glob.glob(in_dir + type)
        video_paths.extend(paths)

    print("Paths: ", len(video_paths))

    # Sử dụng tính toán đa luồng
    for path in video_paths:
        extract_face(path)

    print(len(os.listdir("/mnt/disk1/doan/phucnp/Dataset/ff/video/manipulated_sequences/FaceSwap/image/train")))
    

/mnt/disk1/anaconda3/envs/phucnp/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


['/mnt/disk1/doan/phucnp/Graduation_Thesis/my_thesis/forensics/preprocess_data', '/mnt/disk1/anaconda3/envs/phucnp/lib/python38.zip', '/mnt/disk1/anaconda3/envs/phucnp/lib/python3.8', '/mnt/disk1/anaconda3/envs/phucnp/lib/python3.8/lib-dynload', '', '/mnt/disk1/doan/.local/lib/python3.8/site-packages', '/mnt/disk1/anaconda3/envs/phucnp/lib/python3.8/site-packages']
Paths:  800
Video name:  270_297
Number of taken frame: 12
Video name:  923_023
Number of taken frame: 22
Video name:  776_676
Number of taken frame: 13
Video name:  985_981
Number of taken frame: 13
Video name:  631_551
Number of taken frame: 16
Video name:  262_301
Number of taken frame: 15
Video name:  800_840
Number of taken frame: 13
Video name:  671_677
Number of taken frame: 12
Video name:  937_888
Number of taken frame: 12
Video name:  493_538
Number of taken frame: 15
Video name:  428_466
Number of taken frame: 13
Video name:  514_443
Number of taken frame: 14
Video name:  791_770
Number of taken frame: 13
Video nam